In [2]:
import requests
import pandas as pd
import time


In [3]:

# Define the base URL and API key
api_key = '8265bd1679663a7ea12ac168da84d2e8'
base_url = 'https://api.themoviedb.org/3/movie/top_rated'

# Fetch genre list
genre_url = f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US'
genre_response = requests.get(genre_url)
genres_data = genre_response.json()
genres = {genre['id']: genre['name'] for genre in genres_data['genres']}

# Prepare an empty list to store movie data
all_movies = []

# Loop through all 471 pages
for page in range(1, 472):  # Pages from 1 to 471
    url = f'{base_url}?api_key={api_key}&language=en-US&page={page}'
    
    # Make the request to the API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        movies = data.get('results', [])
        
        # Debug: Check how many movies were retrieved from this page
        print(f"Page {page}: Retrieved {len(movies)} movies.")
        
        # Extract relevant movie details
        for movie in movies:
            # Get the genre names from the genre IDs
            genre_names = [genres[genre_id] for genre_id in movie.get('genre_ids', [])]
            
            all_movies.append({
                'title': movie.get('title'),
                'overview': movie.get('overview'),
                'genres': ', '.join(genre_names)  # Join genres as a comma-separated string
            })
    else:
        print(f"Failed to retrieve data for page {page}: {response.status_code}")
    
    # Sleep for a short duration to avoid hitting rate limits
    time.sleep(0.5)

# Convert the list of dictionaries into a pandas DataFrame


Page 1: Retrieved 20 movies.
Page 2: Retrieved 20 movies.
Page 3: Retrieved 20 movies.
Page 4: Retrieved 20 movies.
Page 5: Retrieved 20 movies.
Page 6: Retrieved 20 movies.
Page 7: Retrieved 20 movies.
Page 8: Retrieved 20 movies.
Page 9: Retrieved 20 movies.
Page 10: Retrieved 20 movies.
Page 11: Retrieved 20 movies.
Page 12: Retrieved 20 movies.
Page 13: Retrieved 20 movies.
Page 14: Retrieved 20 movies.
Page 15: Retrieved 20 movies.
Page 16: Retrieved 20 movies.
Page 17: Retrieved 20 movies.
Page 18: Retrieved 20 movies.
Page 19: Retrieved 20 movies.
Page 20: Retrieved 20 movies.
Page 21: Retrieved 20 movies.
Page 22: Retrieved 20 movies.
Page 23: Retrieved 20 movies.
Page 24: Retrieved 20 movies.
Page 25: Retrieved 20 movies.
Page 26: Retrieved 20 movies.
Page 27: Retrieved 20 movies.
Page 28: Retrieved 20 movies.
Page 29: Retrieved 20 movies.
Page 30: Retrieved 20 movies.
Page 31: Retrieved 20 movies.
Page 32: Retrieved 20 movies.
Page 33: Retrieved 20 movies.
Page 34: Retrieved 

In [5]:
df = pd.DataFrame(all_movies)

df.to_csv('top_rated_movies_full.csv', index=False)
# print(f"Total number of movies retrieved: {len(df)}")


In [6]:
df = pd.read_csv('top_rated_movies_full.csv')

In [7]:
df

,title,overview,genres
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"Drama, Crime"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama, Crime"
2,The Wild Robot,"After a shipwreck, an intelligent robot called...","Animation, Science Fiction, Family"
3,The Godfather Part II,In the continuing saga of the Corleone crime f...,"Drama, Crime"
4,Schindler's List,The true story of how businessman Oskar Schind...,"Drama, History, War"
...,...,...,...
9415,Open Windows,Nick is excited to discover he's won a dinner ...,"Thriller, Action, Crime"
9416,Vampire in Brooklyn,Detective Rita Veder is assigned to a baffling...,"Comedy, Horror, Romance"
9417,Bushwick,When an unknown military force invades Bushwic...,"Thriller, Action"
9418,A Wrinkle in Time,After the disappearance of her scientist fathe...,"Adventure, Science Fiction, Family, Fantasy"


In [8]:
df['overview'][45].lower()

'under the direction of a ruthless instructor, a talented young drummer begins to pursue perfection at any cost, even his humanity.'

In [9]:
df['title'] = df['title'].str.lower()

In [10]:
df['overview'] = df['overview'].str.lower()

In [11]:
df['genres'] = df['genres'].str.lower()

In [12]:
df.dropna(inplace=True)

In [13]:
df['overview'].isnull().sum()

0

In [14]:
df['overview'][4]

'the true story of how businessman oskar schindler saved over a thousand jewish lives from the nazis while they worked as slaves in his factory during world war ii.'

# remove htmls

In [15]:
import re
def remove_html(text):
    sentence = re.compile('<.*?>')
    return sentence.sub(r'',text)

In [16]:
text = "<html><body><p> Movie 1</p><p> Actor - Aamir Khan</p><p> Click here to <a href='http://google.com'>download</a></p></body></html>"

In [17]:
remove_html(text)

' Movie 1 Actor - Aamir Khan Click here to download'

In [18]:
df['overview'] = df['overview'].apply(remove_html)

In [19]:
len(df['overview'])

9419

# remove URL

In [20]:
def remove_url(text):
    sentence = re.compile(r'https?://\S+|www\.\S+')
    return sentence.sub(r'',text)

In [21]:
remove_url(text)

"<html><body><p> Movie 1</p><p> Actor - Aamir Khan</p><p> Click here to <a href='"

In [22]:
df['overview'] = df['overview'].apply(remove_url)

# remove punctuation 

In [23]:
import string ,time
exclude = string.punctuation

In [24]:
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [25]:
def remove_punct(text):
    for char in exclude:
        text = text.replace(char , '')
    return text

In [26]:
remove_punct(text)

'htmlbodyp Movie 1pp Actor  Aamir Khanpp Click here to a hrefhttpgooglecomdownloadapbodyhtml'

In [27]:
df['overview'] = df['overview'].apply(remove_punct)

In [28]:
start = time.time()
remove_punct(text)
time1 = time.time() - start
print(time1*9419)

0.42218494415283203


In [29]:
def reduce_time(text):
    return text.translate(str.make_trans('','',exclude))

In [30]:
start = time.time()
remove_punct(text)
time1 = time.time() - start
print(time1*9419)

0.3795173168182373


In [31]:
from textblob import TextBlob

In [32]:
incorrect_text = 'ceertain conditionas duriing seveal ggenerations aree moodified in the saame maner.'
textblb = TextBlob(incorrect_text)
textblb.correct().string

'certain conditions during several generations are modified in the same manner.'

In [33]:
from nltk.corpus import stopwords

In [34]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [35]:
def remove_words(text):
    new_text = []
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [36]:
remove_words('the defense and the prosecution have rested and the jury is filing into the jury room to decide if a young spanishamerican is guilty or innocent of murdering his father what begins as an open and shut case soon becomes a minidrama of each of the jurors prejudices and preconceptions about the trial the accused and each other')

' defense   prosecution  rested   jury  filing   jury room  decide   young spanishamerican  guilty  innocent  murdering  father  begins   open  shut case soon becomes  minidrama     jurors prejudices  preconceptions   trial  accused   '

In [37]:
df['overview'] = df['overview'].apply(remove_words)

In [38]:
df.head()

,title,overview,genres
0,the shawshank redemption,imprisoned 1940s double murder wife lo...,"drama, crime"
1,the godfather,spanning years 1945 1955 chronicle fictio...,"drama, crime"
2,the wild robot,shipwreck intelligent robot called roz str...,"animation, science fiction, family"
3,the godfather part ii,continuing saga corleone crime family you...,"drama, crime"
4,schindler's list,true story businessman oskar schindler save...,"drama, history, war"


In [39]:
import re
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [40]:
remove_emoji('hey this is so lovely,❤️❤️❤️💛💛 i loved it ')

'hey this is so lovely, i loved it '

# Splitting the sentences

# using the spliting function

In [41]:
sent1 = 'i am going to delhi!'
sent1.split()

['i', 'am', 'going', 'to', 'delhi!']

In [42]:
sent2 = 'i am going to delhi. i will return after two days till then you wait for us to go alwar.after that we\'ll go kahirthal together.'

sent2.split('.')


['i am going to delhi',
 ' i will return after two days till then you wait for us to go alwar',
 "after that we'll go kahirthal together",
 '']

In [43]:
sent3 = 'what i have to do in this situation?.we should either go home or take some action, tell us '
sent3.split('?')

['what i have to do in this situation',
 '.we should either go home or take some action, tell us ']

# Regular Expression

In [44]:
import re 
sent1 = 'i am going to delhi'
token = re.findall("[\w']+" , sent1)
token

['i', 'am', 'going', 'to', 'delhi']

In [45]:
text = """Lorem Ipsum is simply dummy text of the printing and typesetting industry? 
Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, 
when an unknown printer took a galley of type and scrambled it to make a type specimen book."""

sentence = re.compile('[.!?]').split(text)
sentence

['Lorem Ipsum is simply dummy text of the printing and typesetting industry',
 " \nLorem Ipsum has been the industry's standard dummy text ever since the 1500s, \nwhen an unknown printer took a galley of type and scrambled it to make a type specimen book",
 '']

In [46]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [47]:
sent1 = 'I am going to visit delhi!'
word_tokenize(sent1)

['I', 'am', 'going', 'to', 'visit', 'delhi', '!']

In [48]:
text = """Lorem Ipsum is simply dummy text of the printing and typesetting industry? 
Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, 
when an unknown printer took a galley of type and scrambled it to make a type specimen book."""
sent_tokenize(text)

['Lorem Ipsum is simply dummy text of the printing and typesetting industry?',
 "Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, \nwhen an unknown printer took a galley of type and scrambled it to make a type specimen book."]

In [49]:
sent5 = 'I have a Ph.D in A.I'
sent6 = "We're here to help! mail us at nks@gmail.com"
sent7 = 'A 5km ride cost $10.50'
word_tokenize(sent5)

['I', 'have', 'a', 'Ph.D', 'in', 'A.I']

In [50]:
word_tokenize(sent6)

['We',
 "'re",
 'here',
 'to',
 'help',
 '!',
 'mail',
 'us',
 'at',
 'nks',
 '@',
 'gmail.com']

In [51]:
# import spacy

# # Load the English model
# nlp = spacy.load('en_core_web_sm')

# # Example usage
# doc = nlp("I am going to visit Delhi!")
# for token in doc:
#     print(token.text, token.pos_)



In [52]:
from nltk.stem.porter import PorterStemmer

In [53]:
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [54]:
sample = "do did doing done"
stem_words(sample)

'do did do done'

In [55]:
text = 'probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble cause but its not preachy or boring it just never gets old despite my having seen it some 15 or more times in the last 25 years paul lukas performance brings tears to my eyes and bette davis in one of her very few truly sympathetic roles is a delight the kids are as grandma says more like dressedup midgets than children but that only makes them more fun to watch and the mothers slow awakening to whats happening in the world and under her own roof is believable and startling if i had a dozen thumbs theyd all be up for this movie'
print(text)

probably my alltime favorite movie a story of selflessness sacrifice and dedication to a noble cause but its not preachy or boring it just never gets old despite my having seen it some 15 or more times in the last 25 years paul lukas performance brings tears to my eyes and bette davis in one of her very few truly sympathetic roles is a delight the kids are as grandma says more like dressedup midgets than children but that only makes them more fun to watch and the mothers slow awakening to whats happening in the world and under her own roof is believable and startling if i had a dozen thumbs theyd all be up for this movie


In [56]:
stem_words(text)

'probabl my alltim favorit movi a stori of selfless sacrific and dedic to a nobl caus but it not preachi or bore it just never get old despit my have seen it some 15 or more time in the last 25 year paul luka perform bring tear to my eye and bett davi in one of her veri few truli sympathet role is a delight the kid are as grandma say more like dressedup midget than children but that onli make them more fun to watch and the mother slow awaken to what happen in the world and under her own roof is believ and startl if i had a dozen thumb theyd all be up for thi movi'

In [57]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

sentence = "probabl my alltim favorit movi a stori of selfless sacrific and dedic to a nobl caus but it not preachi or bore it just never get old despit my have seen it some 15 or more time in the last 25 year paul luka perform bring tear to my eye and bett davi in one of her veri few truli sympathet role is."
punctuations="?:!.,;"
sentence_words = nltk.word_tokenize(sentence)
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

sentence_words
print("{0:20}{1:20}".format("Word","Lemma"))
for word in sentence_words:
    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word,pos='v')))

Word                Lemma               
probabl             probabl             
my                  my                  
alltim              alltim              
favorit             favorit             
movi                movi                
a                   a                   
stori               stori               
of                  of                  
selfless            selfless            
sacrific            sacrific            
and                 and                 
dedic               dedic               
to                  to                  
a                   a                   
nobl                nobl                
caus                caus                
but                 but                 
it                  it                  
not                 not                 
preachi             preachi             
or                  or                  
bore                bore                
it                  it                  
just            